In [3]:
# ==== CELL 1: Install libraries and Cloudflare tunnel binary ====

!pip install -q diffusers transformers accelerate safetensors \
               streamlit pillow

# Download cloudflared binary for tunneling Streamlit
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

print("✅ Installed: diffusers, transformers, accelerate, safetensors, streamlit, pillow, cloudflared.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 143.7 MB/s eta 0:00:00
✅ Installed: diffusers, transformers, accelerate, safetensors, streamlit, pillow, cloudflared.


In [5]:
%%writefile generator.py
import os
import json
from datetime import datetime

import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

# ---------- Global config ----------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIPE = None
CURRENT_MODEL_ID = None

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Style presets for prompt engineering
STYLE_PRESETS = {
    "None (raw prompt)": "",
    "Photorealistic": "highly detailed, 8k, ultra realistic, professional photography, sharp focus, rich lighting",
    "Digital Art": "digital art, concept art, highly detailed, trending on artstation, 4k illustration",
    "Oil Painting": "oil painting, rich brush strokes, dramatic lighting, classic art style",
    "Cartoon / Anime": "anime style, vibrant colors, cel shading, clean lines, highly detailed",
}

DEFAULT_NEGATIVE = (
    "low quality, blurry, pixelated, distorted, bad anatomy, extra limbs, "
    "watermark, text, cropped, worst quality, low resolution"
)


def init_model(model_id: str, hf_token: str):
    """
    Lazily load Stable Diffusion model.
    Uses GPU if available, otherwise CPU.
    """
    global PIPE, CURRENT_MODEL_ID

    if PIPE is not None and CURRENT_MODEL_ID == model_id:
        return PIPE

    if not hf_token or not hf_token.strip():
        raise ValueError(
            "HuggingFace token is required. "
            "Create one at https://huggingface.co/settings/tokens "
            "and paste it in the sidebar."
        )

    print(f"Loading model '{model_id}' on device '{DEVICE}' ...")

    kwargs = {}
    if DEVICE == "cuda":
        kwargs["torch_dtype"] = torch.float16

    # NOTE: model requires license acceptance on HuggingFace
    PIPE = StableDiffusionPipeline.from_pretrained(
        model_id,
        use_auth_token=hf_token,
        **kwargs,
    )

    # Use a faster scheduler
    try:
        PIPE.scheduler = DPMSolverMultistepScheduler.from_config(PIPE.scheduler.config)
    except Exception:
        pass

    PIPE = PIPE.to(DEVICE)

    if DEVICE == "cuda":
        PIPE.enable_attention_slicing()

    CURRENT_MODEL_ID = model_id
    print("✅ Model loaded.")
    return PIPE


def build_prompt(prompt: str, style_preset: str) -> str:
    style_suffix = STYLE_PRESETS.get(style_preset, "")
    if style_suffix:
        return f"{prompt}, {style_suffix}"
    return prompt


def ensure_run_dir() -> str:
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(OUTPUT_DIR, ts)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir


def save_image_with_metadata(img, metadata: dict, run_dir: str,
                             base_filename: str, index: int,
                             output_formats):
    """
    Save image in PNG/JPEG (as requested) + sidecar JSON with metadata.
    Returns dict of {format: filepath}.
    """
    paths = {}
    for fmt in output_formats:
        fmt = fmt.upper()
        ext = fmt.lower()
        filename = f"{base_filename}_{index+1}.{ext}"
        filepath = os.path.join(run_dir, filename)

        to_save = img
        if fmt == "JPEG":
            to_save = img.convert("RGB")

        to_save.save(filepath, format=fmt)
        paths[fmt] = filepath

    meta_filename = f"{base_filename}_{index+1}_meta.json"
    meta_path = os.path.join(run_dir, meta_filename)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2)

    return paths


def generate_images(
    prompt: str,
    negative_prompt: str,
    num_images: int,
    style_preset: str,
    guidance_scale: float,
    num_inference_steps: int,
    seed: int | None,
    hf_token: str,
    model_id: str,
    base_filename: str,
    output_formats,
):
    """
    Main API called from Streamlit.
    Returns list of dicts: { 'pil_image': img, 'paths': {...}, 'metadata': {...} }
    """
    pipe = init_model(model_id=model_id, hf_token=hf_token)

    full_prompt = build_prompt(prompt, style_preset)
    neg_prompt = negative_prompt.strip() if negative_prompt.strip() else DEFAULT_NEGATIVE

    # Seed / randomness
    generator = None
    if seed is not None:
        generator = torch.Generator(device=DEVICE).manual_seed(int(seed))

    run_dir = ensure_run_dir()

    # Generate images
    result = pipe(
        full_prompt,
        negative_prompt=neg_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        num_images_per_prompt=num_images,
        generator=generator,
    )

    images = result.images

    outputs = []
    for idx, img in enumerate(images):
        meta = {
            "prompt": prompt,
            "full_prompt": full_prompt,
            "negative_prompt": neg_prompt,
            "style_preset": style_preset,
            "guidance_scale": guidance_scale,
            "num_inference_steps": num_inference_steps,
            "seed": seed,
            "model_id": model_id,
            "index": idx,
            "device": DEVICE,
            "timestamp": datetime.now().isoformat(),
        }
        paths = save_image_with_metadata(
            img,
            meta,
            run_dir=run_dir,
            base_filename=base_filename,
            index=idx,
            output_formats=output_formats,
        )
        outputs.append({"pil_image": img, "paths": paths, "metadata": meta})

    return outputs

Overwriting generator.py


In [6]:
%%writefile app.py
import io

import streamlit as st

from generator import (
    generate_images,
    STYLE_PRESETS,
)

st.set_page_config(
    page_title="AI Image Generator (Stable Diffusion)",
    page_icon="🎨",
    layout="wide",
)

st.title("🎨 AI Image Generator (Stable Diffusion)")
st.write("Running on **Google Colab** using **open-source Stable Diffusion (Diffusers)**.")

st.markdown("---")

# ---------- Sidebar: configuration ----------
st.sidebar.header("⚙️ Generation Settings")

hf_token = st.sidebar.text_input(
    "HuggingFace Access Token",
    type="password",
    help="Required for loading Stable Diffusion from HuggingFace Hub.",
)

model_id = st.sidebar.text_input(
    "Model ID",
    value="runwayml/stable-diffusion-v1-5",
    help="Any compatible Stable Diffusion text-to-image model on HuggingFace.",
)

num_images = st.sidebar.slider(
    "Number of images per prompt",
    min_value=1,
    max_value=4,
    value=1,
)

style_preset = st.sidebar.selectbox(
    "Style preset",
    list(STYLE_PRESETS.keys()),
    index=1,  # Photorealistic by default
)

guidance_scale = st.sidebar.slider(
    "Guidance scale (prompt strength)",
    min_value=3.0,
    max_value=15.0,
    value=7.5,
    step=0.5,
    help="Higher = follow text more strongly, but may reduce creativity.",
)

num_steps = st.sidebar.slider(
    "Diffusion steps",
    min_value=15,
    max_value=60,
    value=30,
    step=5,
    help="More steps = better quality but slower.",
)

seed_value = st.sidebar.number_input(
    "Seed (-1 for random)",
    value=-1,
    step=1,
    help="Use a fixed seed for reproducible images. -1 = random each time.",
)

output_formats = st.sidebar.multiselect(
    "Save formats",
    options=["PNG", "JPEG"],
    default=["PNG", "JPEG"],
)

base_filename = st.sidebar.text_input(
    "Base filename (for saving)",
    value="generated_image",
)

estimated_time = num_steps * 0.25  # rough, seconds on GPU
st.sidebar.caption(
    f"⏱️ Estimated time: ~{estimated_time:.1f} sec on GPU, slower on CPU."
)

# ---------- Main area ----------
prompt = st.text_area(
    "Enter your image prompt:",
    value="a futuristic city at sunset, cinematic view",
    height=100,
)

negative_prompt = st.text_input(
    "Negative prompt (optional, to avoid unwanted things)",
    value="low quality, blurry, distorted, bad anatomy, watermark, text",
)

col_btn, _ = st.columns([1, 3])
generate_clicked = col_btn.button("🚀 Generate Images")

if generate_clicked:
    if not prompt.strip():
        st.error("Please enter a prompt.")
    elif not hf_token.strip():
        st.error("Please paste your HuggingFace token in the sidebar.")
    elif not output_formats:
        st.error("Select at least one output format (PNG/JPEG) in the sidebar.")
    else:
        real_seed = None if seed_value < 0 else int(seed_value)

        with st.spinner("Generating images... this may take a bit on first run (model download)."):
            try:
                results = generate_images(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    num_images=num_images,
                    style_preset=style_preset,
                    guidance_scale=guidance_scale,
                    num_inference_steps=num_steps,
                    seed=real_seed,
                    hf_token=hf_token,
                    model_id=model_id,
                    base_filename=base_filename,
                    output_formats=output_formats,
                )
            except Exception as e:
                st.error(f"Generation failed: {e}")
            else:
                st.success(f"Generated {len(results)} image(s). Scroll down to view & download.")

                for idx, out in enumerate(results):
                    img = out["pil_image"]
                    meta = out["metadata"]

                    st.markdown(f"### 🖼️ Image {idx+1}")
                    st.image(img, use_column_width=True)

                    # Download buttons for each requested format
                    for fmt in output_formats:
                        buf = io.BytesIO()
                        save_fmt = fmt.upper()
                        img_to_save = img
                        if save_fmt == "JPEG":
                            img_to_save = img.convert("RGB")

                        img_to_save.save(buf, format=save_fmt)
                        st.download_button(
                            label=f"Download Image {idx+1} as {save_fmt}",
                            data=buf.getvalue(),
                            file_name=f"{base_filename}_{idx+1}.{save_fmt.lower()}",
                            mime=f"image/{save_fmt.lower()}",
                            key=f"download_{idx}_{save_fmt}",
                        )

                    with st.expander(f"Metadata for image {idx+1}"):
                        st.json(meta)

Writing app.py


In [7]:
# ==== CELL 4: Start Streamlit server ====

# Kill any previous Streamlit processes
!pkill -f streamlit || echo "No previous Streamlit process."

# Start new app
!streamlit run app.py --server.address 0.0.0.0 --server.port 8501 > logs.txt 2>&1 &

print("✅ Streamlit server started on port 8501. If something looks wrong, check logs in the next cell.")

^C
✅ Streamlit server started on port 8501. If something looks wrong, check logs in the next cell.


In [8]:
# ==== CELL 5: View first 150 lines of logs if debugging ====
!sed -n '1,150p' logs.txt

In [ ]:
# ==== CELL 6: Create public URL with Cloudflare ====
# Run this cell and WAIT until you see a "trycloudflare.com" URL.

!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-28T09:46:14Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-28T09:46:14Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-28T09:46:16Z INF +--------------------------------------------------------------------------------------------+
2025-11-28T09:46:16Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-28T09:46:16Z INF |  https://ref-priest-rome-devices.trycloudflare.com    

In [2]:
!nvidia-smi

Fri Nov 28 09:44:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----